# Práctica: Clasificación con KNN, 1-NN y Naive Bayes

## 1. Dataset

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y
df.head()

## 2. Preprocesamiento: Exploración y Escalado

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled[:5]

## 3. Evaluación y métricas

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def eval_and_print(model, X_train, X_test, y_train, y_test, name="Modelo"):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"=== {name} ===")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, target_names=iris.target_names))
    print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

## 4. Validación Hold-Out 70/30

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

X_tr, X_te, y_tr, y_te = train_test_split(X_scaled, y, test_size=0.30, stratify=y, random_state=42)

# 1-NN
model_1nn = KNeighborsClassifier(n_neighbors=1)
eval_and_print(model_1nn, X_tr, X_te, y_tr, y_te, "1-NN (Hold-Out)")

# KNN con k={3,5,7,9}
for k in [3,5,7,9]:
    model_k = KNeighborsClassifier(n_neighbors=k)
    eval_and_print(model_k, X_tr, X_te, y_tr, y_te, f"KNN k={k} (Hold-Out)")

# Naive Bayes
nb = GaussianNB()
eval_and_print(nb, X_tr, X_te, y_tr, y_te, "Naive Bayes (Hold-Out)")

## 5. Validación Cross-Validation 10-Fold

In [ ]:
from sklearn.model_selection import cross_val_score

models = {
    "1-NN": KNeighborsClassifier(n_neighbors=1),
    "KNN-3": KNeighborsClassifier(n_neighbors=3),
    "KNN-5": KNeighborsClassifier(n_neighbors=5),
    "KNN-7": KNeighborsClassifier(n_neighbors=7),
    "KNN-9": KNeighborsClassifier(n_neighbors=9),
    "GaussianNB": GaussianNB()
}

print("=== 10-Fold Cross-Validation ===")
for name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=10, scoring='accuracy')
    print(f"{name}: mean={scores.mean():.4f}, std={scores.std():.4f}")

## 6. Validación Leave-One-Out

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()

print("=== Leave-One-Out (LOO) ===")
for name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=loo, scoring='accuracy', n_jobs=-1)
    print(f"{name}: mean={scores.mean():.4f}")